# 3. Modélisation - Clustering KMeans

Ce notebook entraîne et évalue le modèle de segmentation client.

**Objectifs :**
- Trouver le nombre optimal de clusters
- Entraîner le modèle KMeans
- Évaluer la qualité du clustering
- Sauvegarder le modèle

**Auteur :** Thomas Mebarki  
**Date :** Janvier 2026

## 3.1 Configuration et imports

In [ ]:
import sys
from pathlib import Path

# Ajouter le répertoire parent au path
sys.path.insert(0, str(Path.cwd().parent))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score

# Modules du projet
from src.config import (
    PROCESSED_DATA_DIR, 
    MODELS_DIR,
    N_CLUSTERS,
    RANDOM_STATE,
    RFM_FEATURES
)
from src.data.loader import load_rfm_data
from src.models.clustering import CustomerSegmenter, find_optimal_clusters
from src.visualization.plots import plot_elbow_curve, plot_silhouette, plot_cluster_distribution

# Configuration
sns.set_style('whitegrid')
%matplotlib inline

print("Modules importés avec succès")

## 3.2 Chargement des données RFM

In [ ]:
# Charger les features RFM
try:
    rfm_df = load_rfm_data(PROCESSED_DATA_DIR / 'customers_rfm.parquet')
except FileNotFoundError:
    rfm_df = load_rfm_data(PROCESSED_DATA_DIR / 'customers_rfm.csv')

print(f"Shape: {rfm_df.shape}")
rfm_df.head()

In [ ]:
# Préparer les features pour le clustering
X = rfm_df[RFM_FEATURES].copy()
print(f"Features pour clustering: {X.columns.tolist()}")
print(f"Shape: {X.shape}")

## 3.3 Recherche du nombre optimal de clusters

Nous utilisons deux méthodes :
1. **Méthode du coude** (Elbow method) : Chercher le "coude" dans la courbe d'inertie
2. **Score de Silhouette** : Maximiser la cohésion intra-cluster

In [ ]:
# Trouver le nombre optimal de clusters
results = find_optimal_clusters(X, k_range=range(2, 11))
results

In [ ]:
# Visualisation
fig = plot_elbow_curve(
    k_values=results['k'].tolist(),
    inertias=results['inertia'].tolist(),
    silhouettes=results['silhouette'].tolist(),
    save_path='../docs/elbow_curve.png'
)
plt.show()

# Meilleur k selon silhouette
best_k = results.loc[results['silhouette'].idxmax(), 'k']
best_silhouette = results['silhouette'].max()
print(f"\nMeilleur k selon Silhouette: {best_k} (score: {best_silhouette:.3f})")

## 3.4 Entraînement du modèle final

Nous choisissons **k=4** clusters basé sur :
- Le score de Silhouette optimal
- L'interprétabilité métier (4 segments distincts)

In [ ]:
# Entraîner le modèle avec CustomerSegmenter
segmenter = CustomerSegmenter(n_clusters=N_CLUSTERS, random_state=RANDOM_STATE)
labels = segmenter.fit_predict(X)

print(f"Modèle entraîné avec {N_CLUSTERS} clusters")
print(f"\nDistribution des segments:")
unique, counts = np.unique(labels, return_counts=True)
for cluster, count in zip(unique, counts):
    print(f"  Cluster {cluster}: {count:,} clients ({count/len(labels)*100:.1f}%)")

## 3.5 Évaluation du modèle

In [ ]:
# Scaling pour les métriques
X_scaled = segmenter.scaler.transform(X)

# Métriques d'évaluation
silhouette = silhouette_score(X_scaled, labels)
calinski = calinski_harabasz_score(X_scaled, labels)
davies = davies_bouldin_score(X_scaled, labels)

print("Métriques de clustering:")
print(f"  Silhouette Score: {silhouette:.4f} (plus proche de 1 = meilleur)")
print(f"  Calinski-Harabasz: {calinski:.2f} (plus élevé = meilleur)")
print(f"  Davies-Bouldin: {davies:.4f} (plus bas = meilleur)")

In [ ]:
# Diagramme de Silhouette
fig = plot_silhouette(X_scaled, labels, save_path='../docs/silhouette_analysis.png')
plt.show()

In [ ]:
# Distribution des segments
fig = plot_cluster_distribution(labels, save_path='../docs/segment_distribution.png')
plt.show()

## 3.6 Centres des clusters

In [ ]:
# Afficher les centres des clusters (valeurs originales)
centers = segmenter.get_cluster_centers(scaled=False)
print("Centres des clusters (valeurs originales):")
centers

In [ ]:
# Résumé des segments avec statistiques
summary = segmenter.get_segment_summary(X)
print("\nRésumé des segments:")
summary

## 3.7 Sauvegarde du modèle

In [ ]:
# Sauvegarder le modèle et le scaler
model_path, scaler_path = segmenter.save(MODELS_DIR)

print(f"Modèle sauvegardé: {model_path}")
print(f"Scaler sauvegardé: {scaler_path}")

In [ ]:
# Vérification du chargement
loaded_segmenter = CustomerSegmenter.load(MODELS_DIR)
test_predictions = loaded_segmenter.predict(X.head(10))
print(f"Test de chargement réussi. Prédictions: {test_predictions}")

## 3.8 Sauvegarde des résultats

In [ ]:
# Ajouter les labels au dataframe RFM
rfm_with_segments = rfm_df.copy()
rfm_with_segments['segment'] = labels

# Mapper les noms de segments
from src.config import SEGMENT_NAMES
rfm_with_segments['segment_name'] = rfm_with_segments['segment'].map(SEGMENT_NAMES)

# Sauvegarder
output_path = PROCESSED_DATA_DIR / 'customers_rfm_segmented.parquet'
rfm_with_segments.to_parquet(output_path)
print(f"Données segmentées sauvegardées: {output_path}")

rfm_with_segments.head(10)

## 3.9 Résumé

**Modèle entraîné :**
- Algorithme : KMeans
- Nombre de clusters : 4
- Silhouette Score : ~0.68

**Segments identifiés :**

| Segment | Description | % Clients |
|---------|-------------|-----------|
| Clients Récents | Achat récent, faible fréquence | ~54% |
| Clients Fidèles | Achats réguliers | ~3% |
| Clients Dormants | Inactifs depuis longtemps | ~40% |
| Clients VIP | Haute valeur | ~3% |

**Fichiers générés :**
- `models/kmeans_model.pkl`
- `models/scaler.pkl`
- `data/processed/customers_rfm_segmented.parquet`

**Prochaine étape :** Analyse des résultats dans le notebook 04.